# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 11 2023 11:16AM,254764,15,9014965,"Mizner Bioscience, LLC",Released
1,Jan 11 2023 10:58AM,254762,18,INNOVA-9018711,Innova Softgel LLC,Released
2,Jan 11 2023 10:40AM,254759,15,9014964,"Mizner Bioscience, LLC",Executing
3,Jan 11 2023 10:40AM,254759,15,9014966,"Mizner Bioscience, LLC",Executing
4,Jan 11 2023 10:40AM,254759,15,9014967,"Mizner Bioscience, LLC",Released
5,Jan 11 2023 10:08AM,254757,10,WT5512 Out,Methapharm-G,Released
6,Jan 11 2023 9:59AM,254756,21,638204,"NBTY Global, Inc.",Released
7,Jan 11 2023 9:48AM,254755,21,638234,"NBTY Global, Inc.",Released
8,Jan 11 2023 9:47AM,254754,21,638194,"NBTY Global, Inc.",Released
9,Jan 11 2023 9:46AM,254753,10,MSP216998,Methapharm-G,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,254757,Released,1
28,254759,Executing,2
29,254759,Released,1
30,254762,Released,1
31,254764,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254756,NaN,NaN,1.0
254757,NaN,NaN,1.0
254759,NaN,2.0,1.0
254762,NaN,NaN,1.0
254764,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254657,0.0,1.0,0.0
254673,10.0,1.0,0.0
254697,0.0,0.0,1.0
254703,0.0,1.0,0.0
254727,0.0,2.0,27.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254657,0,1,0
254673,10,1,0
254697,0,0,1
254703,0,1,0
254727,0,2,27


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254657,0,1,0
1,254673,10,1,0
2,254697,0,0,1
3,254703,0,1,0
4,254727,0,2,27


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254657,,1,
1,254673,10,1,
2,254697,,,1
3,254703,,1,
4,254727,,2,27


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 11 2023 11:16AM,254764,15,"Mizner Bioscience, LLC"
1,Jan 11 2023 10:58AM,254762,18,Innova Softgel LLC
2,Jan 11 2023 10:40AM,254759,15,"Mizner Bioscience, LLC"
5,Jan 11 2023 10:08AM,254757,10,Methapharm-G
6,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc."
7,Jan 11 2023 9:48AM,254755,21,"NBTY Global, Inc."
8,Jan 11 2023 9:47AM,254754,21,"NBTY Global, Inc."
9,Jan 11 2023 9:46AM,254753,10,Methapharm-G
11,Jan 11 2023 9:36AM,254749,10,Bio-PRF
25,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 11 2023 11:16AM,254764,15,"Mizner Bioscience, LLC",,,1
1,Jan 11 2023 10:58AM,254762,18,Innova Softgel LLC,,,1
2,Jan 11 2023 10:40AM,254759,15,"Mizner Bioscience, LLC",,2,1
3,Jan 11 2023 10:08AM,254757,10,Methapharm-G,,,1
4,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc.",,,1
5,Jan 11 2023 9:48AM,254755,21,"NBTY Global, Inc.",,,1
6,Jan 11 2023 9:47AM,254754,21,"NBTY Global, Inc.",,,1
7,Jan 11 2023 9:46AM,254753,10,Methapharm-G,,,2
8,Jan 11 2023 9:36AM,254749,10,Bio-PRF,,,14
9,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 11:16AM,254764,15,"Mizner Bioscience, LLC",1,,
1,Jan 11 2023 10:58AM,254762,18,Innova Softgel LLC,1,,
2,Jan 11 2023 10:40AM,254759,15,"Mizner Bioscience, LLC",1,2,
3,Jan 11 2023 10:08AM,254757,10,Methapharm-G,1,,
4,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc.",1,,
5,Jan 11 2023 9:48AM,254755,21,"NBTY Global, Inc.",1,,
6,Jan 11 2023 9:47AM,254754,21,"NBTY Global, Inc.",1,,
7,Jan 11 2023 9:46AM,254753,10,Methapharm-G,2,,
8,Jan 11 2023 9:36AM,254749,10,Bio-PRF,14,,
9,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 11:16AM,254764,15,"Mizner Bioscience, LLC",1,,
1,Jan 11 2023 10:58AM,254762,18,Innova Softgel LLC,1,,
2,Jan 11 2023 10:40AM,254759,15,"Mizner Bioscience, LLC",1,2,
3,Jan 11 2023 10:08AM,254757,10,Methapharm-G,1,,
4,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 11:16AM,254764,15,"Mizner Bioscience, LLC",1.0,NaN,NaN
1,Jan 11 2023 10:58AM,254762,18,Innova Softgel LLC,1.0,NaN,NaN
2,Jan 11 2023 10:40AM,254759,15,"Mizner Bioscience, LLC",1.0,2.0,NaN
3,Jan 11 2023 10:08AM,254757,10,Methapharm-G,1.0,NaN,NaN
4,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 11:16AM,254764,15,"Mizner Bioscience, LLC",1.0,0.0,0.0
1,Jan 11 2023 10:58AM,254762,18,Innova Softgel LLC,1.0,0.0,0.0
2,Jan 11 2023 10:40AM,254759,15,"Mizner Bioscience, LLC",1.0,2.0,0.0
3,Jan 11 2023 10:08AM,254757,10,Methapharm-G,1.0,0.0,0.0
4,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3566337,69.0,6.0,10.0
15,509523,2.0,2.0,0.0
16,1018936,3.0,1.0,0.0
18,254762,1.0,0.0,0.0
19,764153,41.0,2.0,0.0
21,1018922,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3566337,69.0,6.0,10.0
1,15,509523,2.0,2.0,0.0
2,16,1018936,3.0,1.0,0.0
3,18,254762,1.0,0.0,0.0
4,19,764153,41.0,2.0,0.0
5,21,1018922,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,69.0,6.0,10.0
1,15,2.0,2.0,0.0
2,16,3.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,41.0,2.0,0.0
5,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,69.0
1,15,Released,2.0
2,16,Released,3.0
3,18,Released,1.0
4,19,Released,41.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,21
Status,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,0.0
Executing,6.0,2.0,1.0,0.0,2.0,1.0
Released,69.0,2.0,3.0,1.0,41.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,2.0,1.0,0.0,2.0,1.0
2,Released,69.0,2.0,3.0,1.0,41.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,2.0,1.0,0.0,2.0,1.0
2,Released,69.0,2.0,3.0,1.0,41.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()